In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import xarray as xr
import xclim as xc
import pyhomogenize as pyh

In [ ]:
ifile='/pool/data/CORDEX/data/cordex/output/EUR-11/GERICS/MPI-M-MPI-ESM-LR/historical/r3i1p1/GERICS-REMO2015/v1/day/tas/v20190925/tas_EUR-11_MPI-M-MPI-ESM-LR_historical_r3i1p1_GERICS-REMO2015_v1_day_20010101-20051231.nc'

In [ ]:
def open_xrdataset(files, use_cftime=True, parallel=True, data_vars='minimal', chunks={'time':1}, 
                   coords='minimal', compat='override', drop=None, **kwargs):
    """optimized function for opening large cf datasets.
    based on https://github.com/pydata/xarray/issues/1385#issuecomment-561920115
    decode_timedelta=False is added to leave variables and coordinates with time units in 
    {“days”, “hours”, “minutes”, “seconds”, “milliseconds”, “microseconds”} encoded as numbers.   
    """
    def drop_all_coords(ds):
        return ds.reset_coords(drop=True)

    ds = xr.open_mfdataset(files, parallel=parallel, decode_times=False, combine='by_coords', 
                           preprocess=drop_all_coords, decode_cf=False, chunks=chunks,
                           data_vars=data_vars, coords=coords, compat=compat, **kwargs)

    return xr.decode_cf(ds, use_cftime=use_cftime, decode_timedelta=False)

In [ ]:
ds = open_xrdataset(ifile)
ds

In [ ]:
_freq = {'year': 'YS',
         'sem' : 'QS-DEC', 
         'mon' : 'MS',
         'day' : 'D'}

_tfreq = {'year': ['YS','Y'],
          'sem' : ['QS-DEC','Q-FEB'],
          'mon' : ['MS','M'],
          'day' : 'D'}

_bounds = {'day'   : {'start' : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],                       
                      'end'   : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]},            
           'year'  : {'start' : [1],                       
                      'end'   : [12]},            
           'sem'   : {'start' : [3, 6, 9, 12],                       
                      'end'   : [2, 5, 8, 11]},            
           'month' : {'start' : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],                       
                      'end'   : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]},            
           'fx'    : {'start' : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],                       
                      'end'   : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]}} 

_fmt = {'day'   : '%Y%m%d',         
        'year'  : '%Y' ,         
        'month' : '%Y%m' ,         
        'sem'   : '%Y%m' ,         
        'fx'    : '%Y%m%d' }

In [ ]:
import json
with open('../tables/indices.json', 'r') as f:
  ijson = json.load(f)
with open('../tables/xcalc.json', 'r') as f:
  xjson = json.load(f)
with open('../tables/projects.json', 'r') as f:
  pjson = json.load(f)

In [ ]:
import numpy as np

class indices:
    def __init__(self, ds, 
                 index,
                 project,
                 institution,
                 institution_id,
                 var_name=None,
                 freq='year',
                 period=None,
                 base_period_time_range=None,
                 time_range=None, 
                 check_time_axis=True, 
                 crop_time_axis=True, 
                 write=False,
                 ):
        
        self.ds   = ds
        self.CIname = index
        self.project = project
        self.institution = institution
        self.institution_id = institution_id
        self.var_name = var_name
        self.freq = freq
        self.fmt = _fmt[freq]
        self.afmt = _fmt[ds.frequency]
        self.period = period
        self.base_period_time_range = base_period_time_range
        self.time_range = time_range
        self.check_time_axis = check_time_axis
        self.crop_time_axis = crop_time_axis
        self.write = write
        
    def _outname(self):
        drs = {}
        drs['output_fmt']   = pjson['climdex'+self.project]['format']
        drs['output_comps'] = pjson['climdex'+self.project]['components'].split(', ')

        ocomps = []
        for comp in drs['output_comps']:
            if hasattr(self.idx_ds, comp):
                ocomps.append(getattr(self.idx_ds, comp))         
            elif hasattr(self.ds, comp): 
                ocomps.append(getattr(self.ds, comp))
            else:
                ocomps.append('NA')
        return drs['output_fmt'].format(*ocomps)
        
    def _get_time_range_as_str(self, time, fmt):
        basics = pyh.basics()
        ts = basics.date_to_str(time[0], fmt)
        te = basics.date_to_str(time[-1], fmt)
        return [ts, te]
    
    def preprocessing(self):
        time_control = pyh.time_control(self.ds)
        if not self.var_name:
            self.var_name=time_control.get_var_name()
        avail_time = self._get_time_range_as_str(time_control.time, self.afmt)
        
        if self.time_range:
            time_control.select_time_range(self.time_range)
        if self.crop_time_axis:
            time_control.select_limited_time_range(smonth=_bounds[self.freq]['start'], 
                                                   emonth=_bounds[self.freq]['end'])   
        req_time = self._get_time_range_as_str(time_control.time, self.fmt)
        
        if self.check_time_axis:
            time_control.check_timestamps(correct=True)
        self.TimeRange  = req_time
        self.ATimeRange = avail_time
        
        return time_control.ds
    
    def processing(self):
        array = getattr(self, self.CIname)()
        basics = pyh.basics()
        date_range = basics.date_range(start=self.preproc.time.values[0],
                                       end=self.preproc.time.values[-1],
                                       frequency=_tfreq[self.freq])
        array = array.assign_coords({'time' : date_range})
        data_vars = {k: self.preproc.data_vars[k] for k in self.preproc.data_vars.keys() if k not in self.var_name}
        data_vars[self.CIname] = array
        data_vars['time'] = array['time']
        del data_vars['time_bnds']
        idx_ds = xr.Dataset(data_vars=data_vars, 
                            attrs=self.preproc.attrs)
        idx_ds = idx_ds.assign_coords({'time' : np.asarray(idx_ds.time, 'datetime64[ns]')})
        idx_ds = idx_ds.cf.add_bounds('time')
        idx_ds = idx_ds.reset_coords('time_bounds')
        idx_ds['time_bounds'] = idx_ds.time_bounds.transpose()
        return idx_ds.rename({'time_bounds':'time_bnds'})

    def postprocessing(self):

        def adjust_attributes(dictionary, value):
            output = {}
            for key in dictionary.keys():
                if isinstance(dictionary[key], dict):
                    output[key] = adjust_attributes(dictionary[key], value)
                else:
                    output[key] = dictionary[key].format(value)
            return output
        
        json = {}
        json[self.CIname] = ijson[self.CIname]
        json[self.CIname].update(xjson['variable_att'])
        json['global_att'] = xjson['global_att']
        json['global_att'].update(xjson[self.project]['global_att'])       
        self.json = adjust_attributes(json, None)
        
        from ci_netcdfattributes import NetCDFvariableattributes, NetCDFglobalattributes
        
        output = NetCDFvariableattributes(self, self.idx_ds[self.CIname], self.json[self.CIname]).output
        output = NetCDFglobalattributes(self, self.idx_ds, self.json['global_att']).output

        return output
    
    def to_netcdf(self):
        MISSVAL=1e20
        encoding={self.CIname:{'_FillValue':MISSVAL, 'missing_value':MISSVAL}, 
                  'time':{'units': self.ds.time.encoding['units'],
                          'calendar': self.ds.time.encoding['calendar'], 
                          'dtype': self.ds.time.encoding['dtype']},
                  'time_bnds':{'units': self.ds.time_bnds.encoding['units'],
                               'calendar': self.ds.time_bnds.encoding['calendar'], 
                               'dtype': self.ds.time.encoding['dtype']}}
        self.postproc.to_netcdf(self.outname, encoding=encoding, format='NETCDF4', unlimited_dims={'time':True})
        
    def compute(self, output=False):
        self.preproc = self.preprocessing()
        self.idx_ds  = self.processing()
        self.postproc = self.postprocessing()
        write = False
        if output is True:
            self.outname = self._outname()
            write = True
        elif isinstance(output, str):
            self.outname = output
            write = True
        if write: self.to_netcdf()
            
        return self.postproc
    
    def TG(self):
        return xc.atmos.tg_mean(ds=self.preproc, freq=_freq[self.freq])

In [ ]:
tg = indices(ds, index='TG', 
             freq='sem', 
             project='CORDEX', 
             institution='Helmholtz-Zentrum hereon GmbH, Climate Service Center Germany', 
             institution_id='GERICS').compute(output=True)
tg